# Práctica 5: Automatic Differentiation in Python
## AD in tensorflow

In [62]:
import tensorflow as tf

In [63]:
x = tf.Variable(4.0)
print(x)

with tf.GradientTape() as tape:
    y = x**2
    
dy_dx = tape.gradient(y, x)
print(f"Derivada de y respecto a x = {dy_dx}")

# Derivada de orden superior
with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape1:
        y = x**3
    dy_dx = tape1.gradient(y, x)
d2y_dx2 = tape2.gradient(dy_dx, x)
print(f"Derivada de y respecto a x = {dy_dx}")
print(f"Derivada segunda de y respecto a x = {d2y_dx2}")
    

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=4.0>
Derivada de y respecto a x = 8.0
Derivada de y respecto a x = 48.0
Derivada segunda de y respecto a x = 24.0


Consideramos la función $$f(x_1,x_2) = x_1^2(x_1+x_2)$$

In [64]:
from numpy.linalg import eig

x = tf.Variable([2.0, 3.0])

with tf.GradientTape() as tape:
    f = x[0]**2 * (x[0] + x[1])
print(f"Gradiente de f = {tape.gradient(f, x)}")

with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape1:
        f = x[0]**2 * (x[0] + x[1])
    grad_f = tape1.gradient(f, x)
hessian_f = tape2.jacobian(grad_f, x)
print(f"Hessiana de f = \n{hessian_f}")

print(f"Autovalores de hessian_f = {eig(hessian_f.numpy())[0]}")

Gradiente de f = [24.  4.]
Hessiana de f = 
[[18.  4.]
 [ 4.  0.]]
Autovalores de hessian_f = [18.848858  -0.8488578]


Veamos un modelo más complicado 
$$ 
y=x\cdot \omega +b 
$$ 
$\omega$ weight

$b$ bias

In [65]:
del tape

In [66]:
tensor_x = tf.Variable([[1,2,3,4]], dtype=tf.float32, name='x')
print(tensor_x)
tensor_w = tf.Variable(tf.random.uniform((4, 3), minval=-20, maxval=20, 
                       dtype=tf.float32), name='w')
print(tensor_w)
tensor_b = tf.Variable(tf.ones(3, dtype=tf.float32), name='b')
print(tensor_b)

y_label = tf.constant([1, 2, 3], dtype=tf.float32)

<tf.Variable 'x:0' shape=(1, 4) dtype=float32, numpy=array([[1., 2., 3., 4.]], dtype=float32)>
<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
array([[-19.123158 ,  17.638596 , -17.768303 ],
       [-15.578828 ,   1.0675144,  10.889381 ],
       [ 12.224632 ,  19.013626 ,   4.790201 ],
       [-10.125589 , -14.577489 , -12.080727 ]], dtype=float32)>
<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>


$$\text{loss} = \dfrac{1}{3}\sum_{i=1}^3(y_i-y_{\text{label}}^i)^2$$

In [72]:
with tf.GradientTape(persistent=True) as tape:
    tensor_y = tensor_x @ tensor_w + tensor_b
    tensor_loss = tf.reduce_mean((tensor_y - y_label) ** 2)
tensor_dy_dx = tape.gradient(tensor_y, tensor_x) # Gradiente de y respecto de x (derivada de y_i respecto de x_j y sumando)
tensor_jac_dy_dx = tape.jacobian(tensor_y, tensor_x)
tensor_dloss_dy = tape.gradient(tensor_loss, tensor_y)
tensor_dloss_dx = tape.gradient(tensor_loss, tensor_x)

print(f"Gradiente de y respecto de x = {tensor_dy_dx}")
print(f"Jacobiana de y respecto de x = \n{tensor_jac_dy_dx}")
print(f"Gradiente de loss respecto de y = {tensor_dloss_dy}")
print(f"Gradiente de loss respecto de x = {tensor_dloss_dx}")

Gradiente de y respecto de x = [[-19.252865  -3.621932  36.028458 -36.783806]]
Jacobiana de y respecto de x = 
[[[[-19.123158  -15.578828   12.224632  -10.125589 ]]

  [[ 17.638596    1.0675144  19.013626  -14.577489 ]]

  [[-17.768303   10.889381    4.790201  -12.080727 ]]]]
Gradiente de loss respecto de y = [[-36.072853  11.669696 -21.294563]]
Gradiente de loss respecto de x = [[1274.0321   342.54572 -321.09937  452.39783]]
